# Imports

In [6]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# From our Data Directory
from pathlib import Path
data_dir = Path('data')

# Uploading and Cleaning Data

In [5]:
student_data = pd.read_csv('data/student_data.csv', sep = ';' )

In [3]:
print(student_data.columns)

Index(['Marital status', 'Application mode', 'Application order', 'Course',
       'Daytime/evening attendance\t', 'Previous qualification',
       'Previous qualification (grade)', 'Nacionality',
       'Mother's qualification', 'Father's qualification',
       'Mother's occupation', 'Father's occupation', 'Admission grade',
       'Displaced', 'Educational special needs', 'Debtor',
       'Tuition fees up to date', 'Gender', 'Scholarship holder',
       'Age at enrollment', 'International',
       'Curricular units 1st sem (credited)',
       'Curricular units 1st sem (enrolled)',
       'Curricular units 1st sem (evaluations)',
       'Curricular units 1st sem (approved)',
       'Curricular units 1st sem (grade)',
       'Curricular units 1st sem (without evaluations)',
       'Curricular units 2nd sem (credited)',
       'Curricular units 2nd sem (enrolled)',
       'Curricular units 2nd sem (evaluations)',
       'Curricular units 2nd sem (approved)',
       'Curricular units 2nd

In [4]:
# We are going to do an EDA on age at enrollment, debtor (whether the student is going into debt) and admission grade. 
# We will then examine their relationship to our target variable (whether the student dropped out or graduated).

clean_sd = student_data[['Previous qualification (grade)','Debtor', 'Age at enrollment', 
                         'Admission grade', 'Target', 'Tuition fees up to date']]

Previous qualification (grade) & Admission grade is from a scale of 0 - 200



Debtor = 1 means that yes they going into debt



Tuition fees up to date = 1 means that yes they are up to date

In [5]:
clean_sd

,Previous qualification (grade),Debtor,Age at enrollment,Admission grade,Target,Tuition fees up to date
0,122.0,0,20,127.3,Dropout,1
1,160.0,0,19,142.5,Graduate,0
2,122.0,0,19,124.8,Dropout,0
3,122.0,0,20,119.6,Graduate,1
4,100.0,0,45,141.5,Graduate,1
...,...,...,...,...,...,...
4419,125.0,0,19,122.2,Graduate,1
4420,120.0,1,18,119.0,Dropout,0
4421,154.0,0,30,149.5,Dropout,1
4422,180.0,0,20,153.8,Graduate,1
